In [1]:
# https://codereview.stackexchange.com/questions/185389/dropping-rows-from-a-pandas-dataframe-where-some-of-the-columns-have-value-0
# https://stackoverflow.com/questions/22231592/pandas-change-data-type-of-series-to-string
# https://realpython.com/pandas-sort-python/#:~:text=To%20sort%20the%20DataFrame%20based,DataFrame%20sorted%20in%20ascending%20order.
# https://artsdatascience.wordpress.com/2019/12/13/python-%E5%AF%A6%E6%88%B0%E7%AF%87%EF%BC%9A%E7%94%A8apyori-library%E5%8E%BB%E5%81%9A-apriori-algorithm/
import warnings
import pandas as pd
import time
from apyori import apriori
warnings.filterwarnings("ignore",category=DeprecationWarning)
df = pd.read_excel('交易資料集.xlsx',sheet_name=0)
df = df.drop(columns=['ITEM_NO','PRODUCT_TYPE','CUST_ID'],axis=1)
#print(df.head())

In [2]:
df = df[(df[['QUANTITY']] > 0).all(axis=1)]
df['INVOICE_NO'] = df['INVOICE_NO'].astype("string")
df = df.sort_values(by = 'INVOICE_NO')
#print(df.head(20))
#print(df.tail(20))
invoList = df['INVOICE_NO'].tolist()
invoSet = set()
for i in range(len(invoList)):
    invoice = invoList[i]
    if invoice not in invoSet:
        invoSet.add(invoice)
#print(len(invoSet)) #36498

In [3]:
sortedList = sorted(list(invoSet), reverse=False)
tranDict = {}
#addSet = set()
i=0
j=1
for k in range(len(sortedList)):
    #invo = sortedList[k]
    matchedID = df.loc[df['INVOICE_NO'] == sortedList[k]].index
    if len(matchedID) > 1:
        for i in range(len(matchedID)):
            #if matchedID[i] not in addSet:
            tranDict.setdefault(j, [])
            item = df['ITEM_ID'][matchedID[i]]
            tranDict[j].append(item)
        j+=1
    #addSet.add(matchedID[i])

In [4]:
data = list(tranDict.values())
print(len(data))
print(data)

15042
[[15547003, 15245997, 15436261, 15436269, 15471069, 15471068], [15586998, 15547003, 15589015], [15436253, 15471069, 15245997, 15471068], [15245997, 15471069], [15471069, 15245997, 15471068, 15436264], [14810112, 14809864], [15246053, 15238055, 15246052, 15246049, 15246050], [15246052, 15246053, 14789044, 15330988], [14789044, 14787526], [14810112, 14809864], [15312982, 15452141, 14921004], [14809864, 14789044, 14810112, 14787526, 14809864], [15246053, 14809864, 14810112, 14789044, 14787526], [14810112, 14789044, 14787526, 14809864], [15102173, 15331043], [15102173, 15331043], [15331043, 15102173], [15194056, 14671860, 15192100, 14790551, 14675955], [14980086, 14671860, 14675955, 15192100, 14790551, 15194056], [14654748, 15157064, 14671860, 14980086, 15194056, 14481833, 15192100], [15192101, 14790551, 14980086, 14671860, 14654748, 15194056, 13217588, 15192100], [15192100, 14654748, 14481833, 14980086, 15157064, 14731963, 14671860, 15194056], [14980872, 14803086], [14980086, 151921

In [8]:
start = time.time()
association_rules = apriori(data, min_support=0.0019, min_confidence=0.2)
association_results = list(association_rules)
print(len(association_results))
#print(association_results[0])

for item in association_results:
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + str(items[0]) + " -> " + str(items[1]))
    print("Support: " + str(item[1]))
    print("Confidence: " + str(item[2][0][2]))
    #print("Lift: " + str(item[2][0][3]))
    print("=====================================")
    
end = time.time()

234
Rule: 1696 -> 70509
Support: 0.0021273766786331604
Confidence: 0.4848484848484848
Rule: 1697 -> 1698
Support: 0.0025927403270841645
Confidence: 0.4534883720930233
Rule: 1697 -> 1755
Support: 0.0019279351150113017
Confidence: 0.3372093023255814
Rule: 1697 -> 23004
Support: 0.002260337721047733
Confidence: 0.3953488372093023
Rule: 1697 -> 70509
Support: 0.003789389708815317
Confidence: 0.6627906976744186
Rule: 1697 -> 88411
Support: 0.0031910650179497405
Confidence: 0.5581395348837208
Rule: 1697 -> 88676
Support: 0.0019279351150113017
Confidence: 0.3372093023255814
Rule: 1697 -> 135493
Support: 0.001994415636218588
Confidence: 0.3488372093023256
Rule: 1697 -> 135789
Support: 0.0021273766786331604
Confidence: 0.3720930232558139
Rule: 1697 -> 3336149
Support: 0.0019279351150113017
Confidence: 0.3372093023255814
Rule: 1698 -> 1707
Support: 0.001994415636218588
Confidence: 0.25
Rule: 1698 -> 1708
Support: 0.0025927403270841645
Confidence: 0.325
Rule: 1746 -> 1698
Support: 0.0019279351150

In [6]:
# 輸出結果
print("執行時間：%f 秒" % (end - start))

執行時間：0.373497 秒


In [52]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,fpgrowth
from mlxtend.frequent_patterns import association_rules
start = time.time()
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df2 = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(df2, min_support=0.0019,use_colnames=True)
#frequent_itemsets
#data[[]]=data[[]].astype("string")

end = time.time()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
frequent_itemsets 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.002593,(15471068)
1,0.002061,(15452141)
2,0.001994,(14921004)
3,0.004587,(15192100)
4,0.003989,(14671860)
...,...,...
455,0.002659,"(14790248, 14929876)"
456,0.001994,"(15336088, 15280990)"
457,0.002127,"(15336088, 15336004)"
458,0.001928,"(14790248, 15336088)"


In [55]:
# 輸出結果
print("執行時間：%f 秒" % (end - start))

執行時間：0.782376 秒


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
association_rulestb=association_rules(frequent_itemsets, metric="lift", min_threshold=0.0019)
rules = association_rulestb.drop(association_rulestb[association_rulestb.lift <1.0].index)
association_rulestb

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(14671860),(15192100),0.003989,0.004587,0.002659,0.666667,145.333333,0.002641,2.986239
1,(15192100),(14671860),0.004587,0.003989,0.002659,0.579710,145.333333,0.002641,2.369820
2,(14675955),(15192100),0.003523,0.004587,0.002526,0.716981,156.301887,0.002510,3.517125
3,(15192100),(14675955),0.004587,0.003523,0.002526,0.550725,156.301887,0.002510,2.217964
4,(14675955),(14671860),0.003523,0.003989,0.002327,0.660377,165.556604,0.002313,2.932700
...,...,...,...,...,...,...,...,...,...
859,(15336004),(15336088),0.005983,0.004521,0.002127,0.355556,78.650980,0.002100,1.544709
860,(14790248),(15336088),0.007911,0.004521,0.001928,0.243697,53.907316,0.001892,1.316245
861,(15336088),(14790248),0.004521,0.007911,0.001928,0.426471,53.907316,0.001892,1.729796
862,(15419008),(15306063),0.006116,0.004122,0.002260,0.369565,89.661290,0.002235,1.579669


In [57]:
test_result = 'association_rulestb_result.xlsx'
association_rulestb.to_excel(test_result)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
df2 = pd.read_excel('association_rulestb_result.xlsx',sheet_name=0)
df2 = df2.drop(columns=['antecedent support','consequent support','support','confidence','lift','leverage','conviction'],axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
df2=df2.astype("string")
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   864 non-null    string
 1   antecedents  864 non-null    string
 2   consequents  864 non-null    string
dtypes: string(3)
memory usage: 20.4 KB


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [64]:
import re
characters = "frozenset({})"
for x in range(len(characters)):
    df2['antecedents'] = df2['antecedents'].replace(characters[x],"")
    df2['antecedents'] = df2['antecedents'].replace(re.compile(r'[^\w\s]+'), '', regex=True)
df2['antecedents']
df2['antecedents'].head(30)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0              14671860
1              15192100
2              14675955
3              15192100
4              14675955
5              14671860
6              14675955
7              14980086
8              15194056
9              15192100
10             15194056
11             14980086
12             15192100
13             14980086
14             14671860
15             14980086
16    14671860 15192100
17    14671860 14980086
18    15192100 14980086
19             14671860
20             15192100
21             14980086
22             15157064
23             15192100
24             15157064
25             14481833
26             14481833
27             15192100
28                70448
29                70509
Name: antecedents, dtype: string

In [66]:

for x in range(len(characters)):
    df2['consequents'] = df2['consequents'].replace(characters[x],"")
    df2['consequents'] = df2['consequents'].replace(re.compile(r'[^\w\s]+'), '', regex=True)
df2['consequents']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0      15192100
1      14671860
2      15192100
3      14675955
4      14671860
         ...   
859    15336088
860    15336088
861    14790248
862    15306063
863    15419008
Name: consequents, Length: 864, dtype: string

In [70]:
count=int(input("請輸入產品數 :"))
if count ==1:
    test=str(input("請輸入產品 : "))
    fliter = (df2['antecedents']==test)
    consequents=(df2[fliter]['consequents'])
    consequentsList=consequents.tolist()
    print("==================")
    print("推薦產品")
#print(consequentsList)
    for i in range(len(consequentsList)):
        cons = consequentsList[i]
        if len(cons) <= 8:
            print(cons)
elif (count==2):
    test1=str(input("請輸入第一筆產品 : "))
    test2=str(input("請輸入第二筆產品 : "))
    test=test1+" "+test2
    fliter = (df2['antecedents']==test)
    consequents=(df2[fliter]['consequents'])
    consequentsList=consequents.tolist()
    print("==================")
    print("推薦產品")
    print(consequentsList)
    
elif (count==3):
    test1=str(input("請輸入第一筆產品 :"))
    test2=str(input("請輸入第二筆產品 : "))
    test3=str(input("請輸入第三筆產品 : "))
    test=test1+" "+test2+" "+test3
    fliter = (df2['antecedents']==test)
    consequents=(df2[fliter]['consequents'])
    consequentsList=consequents.tolist()
    print("==================")
    print("推薦產品")
    print(consequentsList)
else:
    print("輸入錯誤")

#consequents=(df2[fliter]['consequents'])
#consequentsList=consequents.tolist()
#print(consequentsList)
#for i in range(len(consequentsList)):
    #cons = consequentsList[i]
    #if len(cons) <= 8:
        #print(cons)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


請輸入產品數 :5
輸入錯誤
